<a href="https://colab.research.google.com/github/alimelgon/Big-data-architecture-Practica/blob/master/Scrapy_alicia_hoteles_modificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NOTA: Crea una carpeta en tu drive llamada webcrawling para que te funcione la subida a drive sin modificar nombres de ficheros/carpetas

## Instalar scrapy

In [0]:
!pip install scrapy

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Linkear Google Drive

## Montar Spider Class

In [0]:
import scrapy
import json

class BlogSpider(scrapy.Spider):
    name = 'blogspider'
    # Podeis cambiar la url inicial por otra u otras paginas
    start_urls = ['https://www.secretplaces.es/espana/hoteles']

    def parse(self, response):
        # Aqui scrapeamos los datos y los imprimimos a un fichero
        for hotel in response.css('div.hotel-list-description-wrapper'):
            nombre_hotel = hotel.css('h3 ::text').extract_first().strip()
            tipo_alojamiento = hotel.css('div.hotel-list-type ::text').extract_first().replace("\n", " ").replace(",", "").strip()
            precio = hotel.css('ul.list-hotel-list-info li:nth-child(5) ::text').extract_first()
              
            # Print a un fichero
            print(f"{nombre_hotel}, {tipo_alojamiento}, {precio}", file=filep)

        # no hay paginacion, esto no hace falta
        # for next_page in response.css('a.next-posts-link'):
        #     yield response.follow(next_page, self.parse)


## Ejecutar el Crawler + Scraper

In [0]:
# Podeis cambiar la extension y el nombre del fichero data.txt
filep = open('/content/drive/My Drive/webcrawling/data.csv', 'w')

from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(BlogSpider)
process.start()
filep.close()

2019-09-14 21:06:17 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: scrapybot)
2019-09-14 21:06:17 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.7.0, Python 3.6.8 (default, Jan 14 2019, 11:02:34) - [GCC 8.0.1 20180414 (experimental) [trunk revision 259383]], pyOpenSSL 19.0.0 (OpenSSL 1.1.1c  28 May 2019), cryptography 2.7, Platform Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
2019-09-14 21:06:17 [scrapy.crawler] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2019-09-14 21:06:17 [scrapy.extensions.telnet] INFO: Telnet Password: da3269707cf91a49
2019-09-14 21:06:17 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2019-09-14 21:06:17 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy

## Descargar resultados

In [0]:
from google.colab import files
files.download('/content/drive/My Drive/webcrawling/data.csv')

## Enviar resultados a Google Cloud Storage

In [0]:
# Necesitas saber el id de tu proyecto en la consola cloud de google (esta arriba a la izquierda)
project_id = 'heroic-alpha-252318'

In [0]:
# Importa las librerias necesarias y autenticas con google cloud
from google.colab import auth
auth.authenticate_user()

2019-09-14 21:07:04 [google.auth.transport.requests] DEBUG: Making request: POST https://accounts.google.com/o/oauth2/token
2019-09-14 21:07:04 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): accounts.google.com:443
2019-09-14 21:07:04 [urllib3.connectionpool] DEBUG: https://accounts.google.com:443 "POST /o/oauth2/token HTTP/1.1" 200 None


In [0]:
# Ahora nos traemos lo necesario para cloud storage
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

2019-09-14 21:07:10 [googleapiclient.discovery_cache] WARNING: file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file im

In [0]:
# Ahora necesitas el nombre de tu segmento, para ello crea o busca uno aqui https://console.cloud.google.com/storage/browser
bucket_name = 'dataproc-78c05970-c291-47f3-843a-d313935d5526-europe-west1'

# Tambien puedes definir el nombre del fichero
file_name = 'hoteles.csv'

In [0]:
# Ahora cogemos el fichero del resultado del scrapping y lo subimos al bucket
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/content/drive/My Drive/webcrawling/data.csv', 
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name, 
                                       name=file_name,
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload completo')

2019-09-14 21:07:27 [googleapiclient.discovery] INFO: URL being requested: POST https://www.googleapis.com/upload/storage/v1/b/dataproc-78c05970-c291-47f3-843a-d313935d5526-europe-west1/o?name=hoteles.csv&alt=json&uploadType=resumable
2019-09-14 21:07:27 [google_auth_httplib2] DEBUG: Making request: POST https://accounts.google.com/o/oauth2/token


Upload completo


### Cuando la subida haya finalizado, los datos aparecerán en el explorador de datos almacenados de la consola de Cloud de tu proyecto:




In [0]:
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=heroic-alpha-252318
